# Final Project: Car Make Detection
## Kyle Brooks + Nicolas Decavel-Bueff
This notebook will perform the following things:
1. load in yolov5 which will be used to identify cars from images
2. load in a trained resnet101 model that we train in a separate notebook to identify the make of a car

In [57]:
import torch
import math
import cv2
import torch
import torchvision
import pickle


import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

from collections import Counter
from torch.utils.data import Dataset, DataLoader
from PIL import Image
%matplotlib inline

def get_pred_df(model, frame):
    res = model(frame)    
    tmp_df = res.pandas().xyxy[0] 
    return tmp_df

def get_rects(df):
    # Returns (Image, title, rectangle patch) for drawing
    rects = []
    color_map = {'car':'g'}
    for _, row in df.iterrows():
        if row['confidence'] <= 0.5:
            continue
        xy = row['xmin'], row['ymin']
        width = row['xmax'] - row['xmin']
        height = row['ymax'] - row['ymin']

        rect = patches.Rectangle(xy, width, height, fill=False, color=color_map.get(row['name'],'r'), linewidth=2)
        rects.append(rect)
    
    return rects


def get_img_with_rects(frame, rects):
    fig, ax = plt.subplots(figsize = (6,6))

    # Create a Rectangle patch
    for rect in rects:
        try:
            ax.add_patch(rect)
        except:
            continue
    return ax

#     plt.imshow(frame)
#     plt.show()
    
def crop_resize_cars(pred_df, frame, height, width):
    preprocessed_images = []

    pred_df = pred_df[pred_df['name'] == 'car']
    for _, car in pred_df.iterrows():
        cropped_image = frame[math.floor(car['ymin']):math.ceil(car['ymax']), 
                           math.floor(car['xmin']):math.ceil(car['xmax'])]
        
        resized_img = cv2.resize(cropped_image, (height, width))

        preprocessed_images.append(resized_img)
    
    return preprocessed_images


In [23]:
### Functions to create a DataLoader for our frames
class MyDataset(Dataset):
    def __init__(self, images):
        self.images = images
        
    def __getitem__(self, index):
        image = self.images[index]
        return torchvision.transforms.functional.to_tensor(image)
        
    def __len__(self):
        return len(self.images)
    
def create_loader(preprocessed_images):
    dataset = MyDataset(preprocessed_images)
    loader = DataLoader(dataset, batch_size=1)
    return loader

def extract_makes(loader, model):
    res = []
    for x in loader:
        idx = model(x.permute(0,1,2,3)).argmax().item()
        res.append(makes_so_far[idx])
    return res

In [83]:
def plot_frame_dist(new_frame, car_makes, rects, idx):
    c = Counter(car_makes)
    c = {k: v for k, v in sorted(c.items(), key=lambda item: item[1])}
    
    
    fig, axes = plt.subplots(figsize=(15,5), ncols=2)

    # Create a Rectangle patch
    for rect in rects:
        patch_cpy=copy(rect)
        patch_cpy.axes = None
        patch_cpy.figure = None
        patch_cpy.set_transform(axes[0].transData)
        axes[0].add_patch(patch_cpy)

    axes[0].imshow(frame)

    axes[0].imshow(new_frame)
    axes[0].set_title("Current Frame")
    axes[0].set_xticks([])
    axes[0].set_yticks([])
    
    # res = np.unique(c, return_counts=True)
    axes[1].barh(list(c.keys()), 
                 width=list(c.values()))
    axes[1].set_title("Distribution of Car Makes")
    axes[1].spines['right'].set_visible(False)
    axes[1].spines['top'].set_visible(False)
    yint = range(min(c.values()), math.ceil(max(c.values()))+1)
    axes[1].set_xticks(yint)
    plt.tight_layout()
    plt.savefig(f'demo/index_{idx:010}.png', 
                facecolor='w', 
                    edgecolor='none')
    plt.clf()

In [84]:
### Extract our car detection model
detection_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

### Extract our list of car makes
with open('data/makes', 'rb') as f:
    makes_so_far = pickle.load(f)
    

### Extract our pretrained model
PATH = 'model1'
model = torchvision.models.resnet101()

### Update the final layer
model.fc = nn.Linear(2048, len(makes_so_far))

### Load in our weights that we learned previously
model.load_state_dict(torch.load(PATH))
model.eval();

Using cache found in /Users/nicolasdecavel/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-8-11 torch 1.8.0 CPU

Fusing layers... 
Model Summary: 224 layers, 7266973 parameters, 0 gradients
Adding AutoShape... 


In [103]:
### Now let us iterate through our video every nth frame
n_frame = 10

input_video_path = '../test3.mp4'

cap = cv2.VideoCapture(input_video_path)

idx = 0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret and idx%n_frame == 0:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
        ### Use yolov5 to get bounding boxes
        p_df = get_pred_df(detection_model, frame)
        rects = get_rects(p_df)

        pre_cars = crop_resize_cars(p_df, new_frame, 256, 256)
        dl = create_loader(pre_cars)
        car_makes = extract_makes(dl, model)
        
        plot_frame_dist(frame, car_makes, rects, idx+1)        
    elif not ret:
        break
    idx += 1
    

cap.release()

<ipython-input-83-aadd68baf0fe>:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axes = plt.subplots(figsize=(15,5), ncols=2)


<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

...

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

<Figure size 1080x360 with 0 Axes>

In [106]:
import glob
files = []
for g in glob.glob('demo/*.png'):
    files.append(str(g))
    
files = sorted(files)


In [107]:
import imageio 

with imageio.get_writer('mygif.gif', mode='I',  duration=0.5) as writer:
    for filename in files:
        image = imageio.imread(filename)
        writer.append_data(image)
